## Split dataset

Testing dataset: Cam 2 at 220430 220508 & Cam 4 at 220502 & 220510

In [3]:
import os
import numpy as np
import pandas as pd
import json 

In [4]:
root = os.getcwd() + '/../data/birds/annotations'
pathin = root + '/coco_annotations.json'
date_list = ['220430', '220508', '220502','220510']

with open(pathin, "r") as f:
    data = json.load(f)

In [56]:
filenames = [i['file_name'].split('/')[-1] for i in data['images']]
img_id = [i['id'] for i in data['images']]
dates = [f.split('_')[-2] for f in filenames]
cond = [d in date_list for d in dates]

## retrieve image id
test_img_id = np.array(img_id)[cond]
train_img_id = np.array(img_id)[np.invert(cond)]

def get_new_dataset(data, test_img_id):

    img_list = [d for d in data['images'] if d['id'] in test_img_id]
    anno_list = [d for d in data['annotations'] if d['image_id'] in test_img_id]

    data_test = {}
    data_test['images'] = img_list
    data_test['annotations'] = anno_list
    data_test['categories'] = data['categories']

    return data_test

## create new datasets
data_test = get_new_dataset(data, test_img_id)
data_train = get_new_dataset(data, train_img_id)

In [59]:
## save new datasets
with open(root + '/coco_test_dataset.json', "w") as f:
    json_string = json.dumps(data_test)
    f.write(json_string)

In [60]:
with open(root + '/coco_train_dataset.json', "w") as f:
    json_string = json.dumps(data_train)
    f.write(json_string)

Split Train/Validation

In [24]:
with open(root + '/coco_train_dataset.json', "r") as f:
    data_train = json.load(f)

In [25]:
num_val = int(len(data['images'])*0.1)
train_id = [d['id'] for d in data_train['images']]
np.random.shuffle(train_id)
val_id = train_id[:num_val]
id_left = train_id[num_val:]

In [28]:
cond = [d in val_id for d in train_id]

## create new image list
img_val = list(np.array(data_train['images'])[cond])
img_train = list(np.array(data_train['images'])[np.invert(cond)])

## create new annotations list
anno_val = [a for a in data_train['annotations'] if a['image_id'] in val_id]
anno_train = [a for a in data_train['annotations'] if a['image_id'] in id_left]

In [35]:
new_data_val = {}
new_data_val['images'] = img_val
new_data_val['annotations'] = anno_val
new_data_val['categories'] = data_train['categories']

new_data_train = {}
new_data_train['images'] = img_train
new_data_train['annotations'] = anno_train
new_data_train['categories'] = data_train['categories']

In [37]:
with open(root + '/coco_train_dataset.json', "w") as f:
    json_string = json.dumps(new_data_train)
    f.write(json_string)

In [36]:
with open(root + '/coco_val_dataset.json', "w") as f:
    json_string = json.dumps(new_data_val)
    f.write(json_string)